# Рекомендация тарифов


В распоряжении имеются данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. 

Данные прошли проверенную предобработку.

Задача: построить модель с "accuracy" >= 0.75

## Откройте и изучите файл

In [2]:
import pandas as pd
import numpy as np 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split 

In [3]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [4]:
display(df.head())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [5]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

Так как датасет предполагает в себе уже обработанные данные - проверим соотношение интересующих нас ответов и переходим сразу к разбиению данных и моделированию.

In [5]:
print('Процент тарифов Ultra',(df[df['is_ultra'] == 1]['is_ultra'].count()/
       len(df['is_ultra']))*100, '%')

Процент тарифов Ultra 30.647168637212197 %


In [6]:
print('Процент тарифов Smart',(df[df['is_ultra'] == 0]['is_ultra'].count()/
      len(df['is_ultra']))*100, '%')

Процент тарифов Smart 69.35283136278781 %


На основе соотношения тарифов можно утверждать: **Теоретически, модель выставляющая одни нули(Smart'ы), могла бы иметь точность около 70%**


## Разбиение на выборки

In [7]:
features = df.drop(['is_ultra'], axis = 1)
target = df['is_ultra']

В отсутствии скрытой, тестовой, выборки разделим весь набор данных на выборки: для обучения\валидационную\тестовую в соотношениях 6:2:2 

In [8]:
features_train, features_test, target_train, target_test = train_test_split(features, target, 
                                                                            test_size= 0.4, random_state = 12345)
features_test, features_valid, target_test, target_valid = train_test_split(features_test, target_test,
                                                                            test_size = 0.5, random_state = 12345)


In [9]:
print('Размер всей выборки, признаки/ответы :',features.shape,'/', target.shape)
print('Размер обучающей выборки, признаки/ответы:',features_train.shape,'/', target_train.shape)
print('Размер тестовой выборки, признаки/ответы:',features_test.shape,'/', target_test.shape)
print('Размер валидационной выборки, признаки/ответы:',features_valid.shape,'/', target_valid.shape)

Размер всей выборки, признаки/ответы : (3214, 4) / (3214,)
Размер обучающей выборки, признаки/ответы: (1928, 4) / (1928,)
Размер тестовой выборки, признаки/ответы: (643, 4) / (643,)
Размер валидационной выборки, признаки/ответы: (643, 4) / (643,)


## Исследование моделей

Т.к. клиентом предлагается выбор только из двух тарифов - перед нами бинарная классификационная задача, определяющая дальнейший выбор моделей.

Качество подбираемых моделей будем смотреть на валидационной выборке, добавляя каждой модели гиперпараметр - random_state.


* **DecisionTreeClassifier**

Рассмотрим модели с простым перебором по гиперпараметру глубины дерева. 

In [10]:
%%time

best_model_dtc = None
best_result_dtc = 0

for depth in range(1,14):
    model = DecisionTreeClassifier(random_state = 12345, max_depth = depth)
    model.fit(features_train, target_train)
    prediction_valid = model.predict(features_valid)
    ac_test = accuracy_score(target_valid, prediction_valid)
    if ac_test > best_result_dtc:
        best_model_dtc = model
        best_result_dtc = ac_test

print('Лучшая модель решающего дерева:', best_model_dtc)
print('Показатель точности:', best_result_dtc)

Лучшая модель решающего дерева: DecisionTreeClassifier(max_depth=7, random_state=12345)
Показатель точности: 0.7993779160186625
CPU times: user 107 ms, sys: 1.31 ms, total: 108 ms
Wall time: 113 ms


* **RandomForestClassifier**

Рассмотрим модели с переборам по гиперпараметрам: количества деревьев и глубины.


In [11]:
%%time

best_model_rfc = None
best_result_rfc = 0

for est in range(1, 41, 10):
    for depth in range(1,7):
        model = RandomForestClassifier(random_state = 12345, max_depth = depth, n_estimators = est)
        model.fit(features_train, target_train)
        prediction_valid = model.predict(features_valid)
        ac_test = accuracy_score(target_valid, prediction_valid)
        if ac_test > best_result_rfc:
            best_model_rfc = model
            best_result_rfc = ac_test
            
print('Лучшая модель случайного леса:', best_model_rfc)
print('Показатель точности:', best_result_rfc)

Лучшая модель случайного леса: RandomForestClassifier(max_depth=6, n_estimators=21, random_state=12345)
Показатель точности: 0.7978227060653188
CPU times: user 1.03 s, sys: 8.26 ms, total: 1.03 s
Wall time: 1.04 s


* **LogisticRegression**

In [12]:
%%time

model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model.fit(features_train, target_train)
prediction_valid = model.predict(features_valid)
ac_test_lr = model.score(features_valid, target_valid)

print('Показатель точности у модели логической регрессии:',
    ac_test_lr)

Показатель точности у модели логической регрессии: 0.6842923794712286
CPU times: user 28.2 ms, sys: 8.03 ms, total: 36.2 ms
Wall time: 33.8 ms


**Вывод:**

DecisionTreeClassifier - показала наилучшую точность в **79,9%**


RandomForestClassifier - показала точность **79,7%**


LogisticRegression - показала точность **68%** 

## Проверьте модель на тестовой выборке

**Среди сравнительно одинаковых результатов между моделями Решающего дерева и Рандомного леса, в данном случае, отдадим предпочтение решающему дереву т.к. при идентичных показателях точности, модель решающего дерева работает быстрее.**


**Воспроизведём её на тестовой выборке:**

In [13]:
%%time

model = DecisionTreeClassifier(random_state = 12345, max_depth = 7)
model.fit(features_train, target_train)
prediction_test = model.predict(features_test)
accuracy_dtc = accuracy_score(target_test, prediction_test)

print('Точность итоговой модели на тестовой выборке:', accuracy_dtc)

Точность итоговой модели на тестовой выборке: 0.7822706065318819
CPU times: user 4.04 ms, sys: 7.71 ms, total: 11.8 ms
Wall time: 10.1 ms


**Точность модели на тестовой выборке равна 78.2%**

## Проверка моделей на адекватность 

Для данной проверки мы не будем использовать какую-либо конкретную модель, нам достаточно обратиться к начальному соотношению ответов в выборках и использовать условную модель, выдающую только один ответ, охватывающий большую часть тестовой выборки.

In [14]:
print(target_test.value_counts([0]))

0    0.706065
1    0.293935
Name: is_ultra, dtype: float64


Таким образом - адекватной моделью можно считать только ту, превосходящую точностью 70% модели, которой даже делать ничего не нужно, как предлагать всем клиентам тариф смарт.

Построим нашу модель ещё раз.

In [15]:
%%time

model = DecisionTreeClassifier(random_state = 12345, max_depth = 7)
model.fit(features_train, target_train)
prediction_test = model.predict(features_test)
accuracy_dtc = accuracy_score(target_test, prediction_test)

print('Точность итоговой модели на тестовой выборке:', accuracy_dtc)

Точность итоговой модели на тестовой выборке: 0.7822706065318819
CPU times: user 9.57 ms, sys: 283 µs, total: 9.85 ms
Wall time: 9.22 ms


0.78 > 0.70

**Вывод:**

**Точность итоговой модели больше, чем точность случайной модели**

**-> Модель адекватна**

## Общий вывод:

* На заранее предобработанных данных, для выполнения задачи классификации были рассмотрены следующие модели: **DecisionTreeClassifier** ; **RandomForestClassifier** ; **LogisticRegression**.
* Наиболее эффективной была признана модель: **DecisionTreeClassifier**, с гиперпараметрами: **random_state = 12345**, **max_depth = 7**.
* На тестовой выборке модель показала **точность 78,2%**, и **прошла тест на адекватность**.